<a href="https://colab.research.google.com/github/skbetz54/Samuel_DATA606/blob/main/Notebooks/Web_Scrape_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install contractions # used to get rid of contractions (in real-world news articles)
!pip install textblob # used for quantifying polarity of the tweets in the dataset.
!pip install newspaper3k

import pandas as pd
import lxml
from lxml.html import fromstring, tostring

from newspaper import Article
import newspaper

import os
import re

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torchtext
import pickle
import contractions

#Preprocessing Functions
from sklearn.utils import shuffle
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet, stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('punkt') # punkt is a dependency used for nltk's  function "word_tokenize" that does not come when you import nltk
nltk.download('stopwords') # NLTK has a built-in dictionary of stop words that will be used when removing stopwords from the data
nltk.download('averaged_perceptron_tagger') # package that assigns parts of speech, to be used in lemmatization
nltk.download('wordnet') # Added because the above import from corpus sometimes does not work.
from nltk.tag import pos_tag # Another package used in lemmatization to import parts of speech for words

from wordcloud import WordCloud # EDA Tool
from textblob import TextBlob

from google.colab import files, drive

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
# Mounting my google drive to upload the necessary csv file from part 1.1

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [29]:
cnn_opinions = 'https://www.cnn.com/opinions/'

opinion_cnn = newspaper.build(cnn_opinions, language='en', memoize_articles=False)

links_cnn_opinion = []

string1 = 'cnn.com'
string2 = 'opinions'

for article in opinion_cnn.articles:
  a = True if string1 in article.url else False
  b = True if string2 in article.url else False
  if a and b ==True:
    links_cnn_opinion.append(article.url)


print(len(links_cnn_opinion))

130


In [30]:
list_cnn = list()

for link in links_cnn_opinion:
  article = Article(link)
  article.download()
  article.parse()
  list_cnn.append({"Title":article.title,
                   "Date":article.publish_date,
                   "URL": article.url,
                   "Text":article.text})
  
df_cnn = pd.DataFrame.from_dict(list_cnn)

df_cnn.head()

,Title,Date,URL,Text
0,Opinion: It won't be long before extremists in...,2022-05-13,https://www.cnn.com/2022/05/13/opinions/gop-ex...,"Julian Zelizer, a CNN political analyst, is a ..."
1,Michael Osterholm: Covid-19 keeps firing 210-m...,2022-05-13,https://www.cnn.com/2022/05/13/opinions/covid-...,Peter Bergen is CNN's national security analys...
2,Opinion: Katie Porter's example reveals a pivo...,2022-05-13,https://www.cnn.com/2022/05/13/opinions/inflat...,(CNN) At a private meeting of the House Democr...
3,Opinion: Why the baby formula shortage is so t...,2022-05-13,https://www.cnn.com/2022/05/13/opinions/baby-f...,"Syra Madad, DHSc, MSc, MCP is an infectious di..."
4,Opinion: Trust in the Supreme Court is grievou...,2022-05-12,https://www.cnn.com/2022/05/12/opinions/suprem...,Barbara A. Perry is the Gerald L. Baliles Prof...


In [ ]:
# for article in page_features.articles:
#   print(article.url)

# Set Up for ML

In [34]:
# Tokenize

df_cnn["Text"] = df_cnn["Text"].str.lower()

df_cnn['tokenized_text'] = df_cnn.apply(lambda row: nltk.word_tokenize(row['Text']), axis = 1)

df_cnn.head()

,Title,Date,URL,Text,tokenized_text
0,Opinion: It won't be long before extremists in...,2022-05-13,https://www.cnn.com/2022/05/13/opinions/gop-ex...,"julian zelizer, a cnn political analyst, is a ...","[julian, zelizer, ,, a, cnn, political, analys..."
1,Michael Osterholm: Covid-19 keeps firing 210-m...,2022-05-13,https://www.cnn.com/2022/05/13/opinions/covid-...,peter bergen is cnn's national security analys...,"[peter, bergen, is, cnn, 's, national, securit..."
2,Opinion: Katie Porter's example reveals a pivo...,2022-05-13,https://www.cnn.com/2022/05/13/opinions/inflat...,(cnn) at a private meeting of the house democr...,"[(, cnn, ), at, a, private, meeting, of, the, ..."
3,Opinion: Why the baby formula shortage is so t...,2022-05-13,https://www.cnn.com/2022/05/13/opinions/baby-f...,"syra madad, dhsc, msc, mcp is an infectious di...","[syra, madad, ,, dhsc, ,, msc, ,, mcp, is, an,..."
4,Opinion: Trust in the Supreme Court is grievou...,2022-05-12,https://www.cnn.com/2022/05/12/opinions/suprem...,barbara a. perry is the gerald l. baliles prof...,"[barbara, a., perry, is, the, gerald, l., bali..."


In [37]:
# Stop Words

df_cnn["Text"] = df_cnn["Text"].str.lower()


regex = re.compile('[!@#$%^&*()?!],.:;\"\'')

# df_cnn['no_stopwords'] = df_cnn['Text'].apply(lambda x: regex.sub("", x)) # Copying our previous step into a new one

stop_words = set(stopwords.words('english')) # Declaring only english words as the ones that we want to use

# Because huggingface has already gotten rid of contractions, I need to add some common contractions WITHOUT the apostrophe so they are still recognized
# NOTE: This will likely not be required in real-world text since the incoming text will not be partially cleaned already.
cont_no_apostrophe = ['dont','id','im','ive','ill','shouldve','couldve','arent','its','itll','itd','thats','thatd','whod','youve','were',
                      'whod','whats','theyre','theres','thered', 'CNN','/n',",","a","'"]
# Adding above words to the set
for word in cont_no_apostrophe:
  stop_words.add(word)


df_cnn['no_stopwords'] = df_cnn['tokenized_text'] # Copying our previous step into a new one

# Finally applying a function to remove all of the 
df_cnn['no_stopwords'] = df_cnn['no_stopwords'].apply(lambda x:[word for word in x if word not in stop_words])

df_cnn.head()



,Title,Date,URL,Text,tokenized_text,no_stopwords
0,Opinion: It won't be long before extremists in...,2022-05-13,https://www.cnn.com/2022/05/13/opinions/gop-ex...,"julian zelizer, a cnn political analyst, is a ...","[julian, zelizer, ,, a, cnn, political, analys...","[julian, zelizer, cnn, political, analyst, pro..."
1,Michael Osterholm: Covid-19 keeps firing 210-m...,2022-05-13,https://www.cnn.com/2022/05/13/opinions/covid-...,peter bergen is cnn's national security analys...,"[peter, bergen, is, cnn, 's, national, securit...","[peter, bergen, cnn, 's, national, security, a..."
2,Opinion: Katie Porter's example reveals a pivo...,2022-05-13,https://www.cnn.com/2022/05/13/opinions/inflat...,(cnn) at a private meeting of the house democr...,"[(, cnn, ), at, a, private, meeting, of, the, ...","[(, cnn, ), private, meeting, house, democrati..."
3,Opinion: Why the baby formula shortage is so t...,2022-05-13,https://www.cnn.com/2022/05/13/opinions/baby-f...,"syra madad, dhsc, msc, mcp is an infectious di...","[syra, madad, ,, dhsc, ,, msc, ,, mcp, is, an,...","[syra, madad, dhsc, msc, mcp, infectious, dise..."
4,Opinion: Trust in the Supreme Court is grievou...,2022-05-12,https://www.cnn.com/2022/05/12/opinions/suprem...,barbara a. perry is the gerald l. baliles prof...,"[barbara, a., perry, is, the, gerald, l., bali...","[barbara, a., perry, gerald, l., baliles, prof..."


In [38]:
# Lemmatize

df_cnn['part_of_speech'] = df_cnn['no_stopwords'].apply(nltk.tag.pos_tag) # Using nltk's built in "pos_tag" to add parts of speech for each token (word)

def wordnet_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return wordnet.NOUN # This part was changed from 'return NONE' because there are several other tags found here not in the above categories, which will be changed to nouns

# Creating new column for the wordnet part of speech to use, applying the above function
df_cnn['wn_part_of_speech'] = df_cnn['part_of_speech'].apply(lambda x: [(word, wordnet_tagger(pos_tag)) for (word, pos_tag) in x])



df_cnn.head()

,Title,Date,URL,Text,tokenized_text,no_stopwords,part_of_speech,wn_part_of_speech
0,Opinion: It won't be long before extremists in...,2022-05-13,https://www.cnn.com/2022/05/13/opinions/gop-ex...,"julian zelizer, a cnn political analyst, is a ...","[julian, zelizer, ,, a, cnn, political, analys...","[julian, zelizer, cnn, political, analyst, pro...","[(julian, JJ), (zelizer, NN), (cnn, JJ), (poli...","[(julian, a), (zelizer, n), (cnn, a), (politic..."
1,Michael Osterholm: Covid-19 keeps firing 210-m...,2022-05-13,https://www.cnn.com/2022/05/13/opinions/covid-...,peter bergen is cnn's national security analys...,"[peter, bergen, is, cnn, 's, national, securit...","[peter, bergen, cnn, 's, national, security, a...","[(peter, NN), (bergen, NN), (cnn, NN), ('s, PO...","[(peter, n), (bergen, n), (cnn, n), ('s, n), (..."
2,Opinion: Katie Porter's example reveals a pivo...,2022-05-13,https://www.cnn.com/2022/05/13/opinions/inflat...,(cnn) at a private meeting of the house democr...,"[(, cnn, ), at, a, private, meeting, of, the, ...","[(, cnn, ), private, meeting, house, democrati...","[((, (), (cnn, NN), (), )), (private, JJ), (me...","[((, n), (cnn, n), (), n), (private, a), (meet..."
3,Opinion: Why the baby formula shortage is so t...,2022-05-13,https://www.cnn.com/2022/05/13/opinions/baby-f...,"syra madad, dhsc, msc, mcp is an infectious di...","[syra, madad, ,, dhsc, ,, msc, ,, mcp, is, an,...","[syra, madad, dhsc, msc, mcp, infectious, dise...","[(syra, NN), (madad, NN), (dhsc, NN), (msc, NN...","[(syra, n), (madad, n), (dhsc, n), (msc, n), (..."
4,Opinion: Trust in the Supreme Court is grievou...,2022-05-12,https://www.cnn.com/2022/05/12/opinions/suprem...,barbara a. perry is the gerald l. baliles prof...,"[barbara, a., perry, is, the, gerald, l., bali...","[barbara, a., perry, gerald, l., baliles, prof...","[(barbara, NN), (a., NN), (perry, NN), (gerald...","[(barbara, n), (a., n), (perry, n), (gerald, v..."


In [39]:
# Lemma String

lemmatizer = WordNetLemmatizer()

df_cnn['lemmatized'] = df_cnn['wn_part_of_speech'].apply(lambda x: [lemmatizer.lemmatize(word, pos) for word, pos in x])

df_cnn['lemma_string'] = [' '.join(map(str,l)) for l in df_cnn['lemmatized']]


In [40]:
df_cnn.head()

,Title,Date,URL,Text,tokenized_text,no_stopwords,part_of_speech,wn_part_of_speech,lemmatized,lemma_string
0,Opinion: It won't be long before extremists in...,2022-05-13,https://www.cnn.com/2022/05/13/opinions/gop-ex...,"julian zelizer, a cnn political analyst, is a ...","[julian, zelizer, ,, a, cnn, political, analys...","[julian, zelizer, cnn, political, analyst, pro...","[(julian, JJ), (zelizer, NN), (cnn, JJ), (poli...","[(julian, a), (zelizer, n), (cnn, a), (politic...","[julian, zelizer, cnn, political, analyst, pro...",julian zelizer cnn political analyst professor...
1,Michael Osterholm: Covid-19 keeps firing 210-m...,2022-05-13,https://www.cnn.com/2022/05/13/opinions/covid-...,peter bergen is cnn's national security analys...,"[peter, bergen, is, cnn, 's, national, securit...","[peter, bergen, cnn, 's, national, security, a...","[(peter, NN), (bergen, NN), (cnn, NN), ('s, PO...","[(peter, n), (bergen, n), (cnn, n), ('s, n), (...","[peter, bergen, cnn, 's, national, security, a...",peter bergen cnn 's national security analyst ...
2,Opinion: Katie Porter's example reveals a pivo...,2022-05-13,https://www.cnn.com/2022/05/13/opinions/inflat...,(cnn) at a private meeting of the house democr...,"[(, cnn, ), at, a, private, meeting, of, the, ...","[(, cnn, ), private, meeting, house, democrati...","[((, (), (cnn, NN), (), )), (private, JJ), (me...","[((, n), (cnn, n), (), n), (private, a), (meet...","[(, cnn, ), private, meeting, house, democrati...",( cnn ) private meeting house democratic caucu...
3,Opinion: Why the baby formula shortage is so t...,2022-05-13,https://www.cnn.com/2022/05/13/opinions/baby-f...,"syra madad, dhsc, msc, mcp is an infectious di...","[syra, madad, ,, dhsc, ,, msc, ,, mcp, is, an,...","[syra, madad, dhsc, msc, mcp, infectious, dise...","[(syra, NN), (madad, NN), (dhsc, NN), (msc, NN...","[(syra, n), (madad, n), (dhsc, n), (msc, n), (...","[syra, madad, dhsc, msc, mcp, infectious, dise...",syra madad dhsc msc mcp infectious disease epi...
4,Opinion: Trust in the Supreme Court is grievou...,2022-05-12,https://www.cnn.com/2022/05/12/opinions/suprem...,barbara a. perry is the gerald l. baliles prof...,"[barbara, a., perry, is, the, gerald, l., bali...","[barbara, a., perry, gerald, l., baliles, prof...","[(barbara, NN), (a., NN), (perry, NN), (gerald...","[(barbara, n), (a., n), (perry, n), (gerald, v...","[barbara, a., perry, gerald, l., baliles, prof...",barbara a. perry gerald l. baliles professor d...


In [41]:
save_path = "/content/gdrive/MyDrive/DATA606/Data/df_cnn_0514.csv"

with open(save_path, 'w', encoding = 'utf-8-sig') as f:
  df_cnn.to_csv(f)